In [6]:
from objects.instance import Instance
from script.data import loader 
import matplotlib
data

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
data = loader('ft06')
inst = Instance(**data)

In [5]:
# Recherche des taches possibles à t
t=0
def recherche_tach(inst,t):
    tache = []
    for j in inst.jobs_list:
        if j.current_task.machine.state=="Free":
            tache.append(j.current_task)
    
    # prio
    # Recherche du max
    for k in range(len(tache)):
        maxi = -1
        for e in tache:
            if e.duration > maxi:
                if e.machine.state == "Free":
                    maxi = e.duration
                    maxi_task = e
        
        maxi_task.update_task_and_date("On going",t)
        maxi_task.machine.update_current_task(maxi_task)
        maxi_task.job.update_all_task()
        try:
            tache.remove(maxi_task)
        except:
            pass
    for j in inst.jobs_list:
        j.update_all_task()

In [8]:
print(inst)

[[2, 1, 0, 3, 1, 6, 3, 7, 5, 3, 4, 6], [1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4], [2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7], [1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9], [2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1], [1, 3, 3, 3, 5, 9, 0, 10, 4, 4, 2, 1]]


In [6]:
recherche_tach(inst,t)

In [7]:
for res in inst.resource_list:
    print(res)

Machine number = 0, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 1, State = Busy, Current Job = 1, Current Task = 0
Machine number = 2, State = Busy, Current Job = 4, Current Task = 0
Machine number = 3, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 4, State = Free, Current Job = N/A, Current Task = N/A
Machine number = 5, State = Free, Current Job = N/A, Current Task = N/A


In [9]:
for i in range(37,40):
    for res in inst.resource_list:
        if res.current_task.finishDate < i:
            print("true")
            print(res.current_job)
            res.state= "Free"
            res.current_job.update_current_task(res.current_task.taskID+1)
            res.current_task.update_task("Done")
            res.current_job.current_task.update_task("On going")
            print(res.current_job)

            recherche_tach(inst,i)

true
Task list
Task # 0 : JobID= 7, taskID= 0, Start= 0, Finish=38, Machine= Machine number = 2, State = Busy, Current Job = 7, Current Task = 0, State= On going
Task # 1 : JobID= 7, taskID= 1, Start= 0, Finish=60, Machine= Machine number = 0, State = Busy, Current Job = 4, Current Task = 0, State= Not Started
Task # 2 : JobID= 7, taskID= 2, Start= 0, Finish=41, Machine= Machine number = 1, State = Busy, Current Job = 3, Current Task = 0, State= Not Started
Task # 3 : JobID= 7, taskID= 3, Start= 0, Finish=24, Machine= Machine number = 3, State = Busy, Current Job = 6, Current Task = 0, State= Not Started
Task # 4 : JobID= 7, taskID= 4, Start= 0, Finish=83, Machine= Machine number = 4, State = Busy, Current Job = 9, Current Task = 0, State= Not Started
Job state= Not Done
Task list
Task # 0 : JobID= 7, taskID= 0, Start= 0, Finish=38, Machine= Machine number = 2, State = Free, Current Job = 7, Current Task = 0, State= Done
Task # 1 : JobID= 7, taskID= 1, Start= 0, Finish=60, Machine= Mac

In [10]:
for re in inst.resource_list:
    print(re)

Machine number = 0, State = Busy, Current Job = 4, Current Task = 0
Machine number = 1, State = Busy, Current Job = 3, Current Task = 0
Machine number = 2, State = Free, Current Job = 7, Current Task = 0
Machine number = 3, State = Busy, Current Job = 6, Current Task = 0
Machine number = 4, State = Busy, Current Job = 9, Current Task = 0


In [4]:
import matplotlib
%matplotlib qt

In [5]:
from objects.instance import Instance
from script.data import loader 
from script.affichage import plot_gantt

data = loader('ft06')
inst = Instance(**data)    # sélectionner l'opération (i,j) de plus grande priorité  

In [7]:
plot_gantt(inst, by="MACHINE")

In [5]:
d1 = {'a': (1,2), 'b': (1,2), 'c': (1,2), 'd': (1,2), 'a1': 5, 'b1': 10, 'c1': 8, 'd1': 6}


In [ ]:
d1

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

In [ ]:
sorted(d1.items(), key=lambda t: t[0])


In [ ]:
d1.items()

{'a': (1, 2),
 'b': (1, 2),
 'c': (1, 2),
 'd': (1, 2),
 'a1': 5,
 'b1': 10,
 'c1': 8,
 'd1': 6}

In [7]:
sorted(d1.items(), key=lambda t: t[0])


[('a', (1, 2)),
 ('a1', 5),
 ('b', (1, 2)),
 ('b1', 10),
 ('c', (1, 2)),
 ('c1', 8),
 ('d', (1, 2)),
 ('d1', 6)]

In [4]:
d1.items()

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', 4), ('a1', 5), ('b1', 10), ('c1', 8), ('d1', 6)])